In [0]:
%%capture
!pip install pyro-ppl 


quote from: https://forum.pyro.ai/t/correct-map-guide-without-using-automatic-guide-generation/940 

**"If you want a MAP estimate for alpha, you need to pyro.sample it from a prior distribution in your model and delta distribution in your guide just like your second guide does with theta, rather than treating it as a constant. If you instead want a maximum likelihood estimate of alpha, you need to wrap it with a pyro.param call in the model and omit it from your guide"**


In [0]:
# General
import os
from collections import defaultdict
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
# Pytorch
import torch
from torch.distributions import constraints
import torchvision.datasets as dset
import torch.nn as nn
import torchvision.transforms as transforms
# Pyro
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO,TraceGraph_ELBO,JitTrace_ELBO
from pyro.optim import Adam
pyro.enable_validation(True)    # <---- This is always a good idea!

In [0]:
data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))

In [0]:
def train(model, guide,lr=0.005):
  # set up the optimizer
  adam_params = {"lr": lr, "betas": (0.90, 0.999)}
  optimizer = Adam(adam_params)
  pyro.clear_param_store()
  # setup the inference algorithm
  svi = SVI(model, guide, optimizer, loss=JitTrace_ELBO())

  n_steps = 5000
  # do gradient steps
  for step in range(n_steps):
      loss = svi.step(torch.tensor(data))
      if step % 1000 == 0:
          print('%.4f' %(loss))

# MLE


In [0]:
def model_MLE(data):
    # define the hyperparameters that control the beta prior

    # sample f from the beta prior
    f = pyro.param("latent_fairness", torch.tensor(0.1), constraint=constraints.positive)
    # loop over the observed data
    with pyro.plate('data', len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

In [0]:
def guide_MLE(data):
  # mu_map = pyro.param('mu_map', torch.tensor(1.0))
  # pyro.sample('theta', dist.Delta(mu_map))
  return None

In [0]:
train(model_MLE,guide_MLE)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  import sys


14.2370
6.7301
6.7301
6.7301
6.7301


In [0]:
pyro.param('latent_fairness').item()

0.5999999642372131

0.599999 very close to 0.6. 

# Posterior

This is nothing special, I copy whole from the tutorial. Prior is Beta(10,10), then the posterior is 0.533333

In [0]:
def model_posterior(data):
    # define the hyperparameters that control the beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    with pyro.plate('data', len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_posterior(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

In [0]:
train(model_posterior,guide_posterior,lr=0.0005)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the 

7.2614
7.0590
7.0529
7.0668
7.0819


In [0]:
import math
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))


based on the data and our prior belief, the fairness of the coin is 0.536 +- 0.090


MAP is the mode of posterior distribution, now or we plot the distribution, or we compute it directly. I choose latter



In [0]:
import math
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()
print((alpha_q-1)/(alpha_q+beta_q-2))

0.5386844444728947


# MAP

When define Delta distribution, please don't init it as 1 for Beta, it will jump to NaN or inf.

In [0]:
def model_map(data):
    f = pyro.sample("latent_fairness", dist.Beta(1, 1))
    with pyro.plate('data', len(data)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_map(data):
    mu_map = pyro.param("mu_map", torch.tensor(0.5),
                         constraint=constraints.positive)
    return pyro.sample("latent_fairness", dist.Delta(mu_map))

In [0]:
train(model_map,guide_map)

6.9315


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


6.7301
6.7301
6.7301
6.7301


In [0]:
map_final = guide_map(data)

In [0]:
map_final

tensor(0.6000, grad_fn=<ExpandBackward>)

Ok beta(1,1) is uniform, then map equal to mle. = 0.6 

In [0]:
def model_map(data):
    f = pyro.sample("latent_fairness", dist.Beta(10, 10))
    with pyro.plate('data', len(data)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

def guide_map(data):
    mu_map = pyro.param("mu_map", torch.tensor(0.5),
                         constraint=constraints.positive)
    return pyro.sample("latent_fairness", dist.Delta(mu_map))

In [0]:
train(model_map,guide_map)
map_final = guide_map(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


5.6719
5.6004
5.6004
5.6004
5.6004


In [0]:
map_final

tensor(0.5357, grad_fn=<ExpandBackward>)

MAP of Beta(10,10) is same as above, great.

What is the real MAP if we compute using our knowledge?

In [0]:
new_alpha = 10 + 6
new_beta = 10 + 4
print((new_alpha-1)/(new_alpha+new_beta-2))

0.5357142857142857


You see, using Delta this time is more accurate than compute whole posterios distribution. Very accurate indeed.